<a href="https://colab.research.google.com/github/RiyaSharma82308/NLP/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("learning NLP here")
#

learning NLP here


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-14acad07-57e8-20ea-884d-7e45a418f33d)


In [4]:
#helper functions for the notebook imported from github
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback , plot_loss_curves,compare_historys

--2023-04-26 18:57:27--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-26 18:57:27 (104 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Get the text dataset


In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
#this data taken from kaggle


--2023-04-26 18:57:31--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   630KB/s    in 0.9s    

2023-04-26 18:57:32 (630 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
unzip_data("nlp_getting_started.zip")   #unzip the data

In [7]:
# visualizing the text dataset

In [8]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
#shuffle training dataframe
train_df_shuffled= train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","(disaster)" if target > 0 else "(not disaster)")
  print(f"text:{text}\n")

Target:1 (disaster)
text:Mexico: construction of bridge collapse killsåÊone http://t.co/I2C00FcOwb http://t.co/jAAgcFaRTW

Target:0 (not disaster)
text:A grade in Black Horse Famine[MEGA]. Score 0840728
http://t.co/pdiit0AF3Q
#Dynamix http://t.co/ZQ5KDOx7BY

Target:0 (not disaster)
text:Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife #islam

Target:0 (not disaster)
text:okay the cat has been quarantined in my bathroom...its meowing really loud but I turned up the TV louder...things just might work out okay

Target:1 (disaster)
text:Wild fires freak me the fuck out. Like hell no



In [13]:
#train test split
from sklearn.model_selection import train_test_split

In [14]:
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [15]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                        train_df_shuffled["target"].to_numpy(),
                                                                        test_size=0.1,
                                                                        random_state=42)

In [16]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [17]:
train_sentences[:3]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....'],
      dtype=object)

In [18]:
#text vectorization(tokenization)
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [19]:
# default parameters
text_vectorizer = TextVectorization(max_tokens=None,
                     standardize="lower_and_strip_punctuation",
                     split="whitespace",
                     ngrams=None,
                     output_mode="int",
                     output_sequence_length=None,
                     pad_to_max_tokens=False)

In [20]:
max_vocab_length=10000
max_length=15
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [21]:
text_vectorizer.adapt(train_sentences)

In [22]:
text_vectorizer(["my name is riya sharma uwhfu igdfiu sduhhue jhd ddugf uugr sudy eugh uyhdu dfuhe jdbv dvj"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 13, 735,   9,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1]])>

In [23]:
type(train_sentences[0])

str

In [24]:
random_sentence=random.choice(train_sentences)
# type(random_sentence)
print(f"original sentence\n{random_sentence}\
\n\n Vectorized version: ")
text_vectorizer([random_sentence])

original sentence
Accident in #Ashville on US 23 SB before SR 752 #traffic http://t.co/hylMo0WgFI

 Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 128,    4,    1,   11,   69, 3215, 3442,  239, 4548,    1,  592,
           1,    0,    0,    0]])>

In [25]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
least_5_words=words_in_vocab[-5:]
print(top_5_words)
print(least_5_words)

['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [26]:
#creating an embedding using an embedding layer

In [27]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim = 128,
                             input_length = max_length)

In [28]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

The X-rays in MKX be looking like fatalities.


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04605692, -0.04459801,  0.01159955, ...,  0.00157427,
          0.00380369, -0.03338087],
        [ 0.00073633,  0.02048076,  0.04580673, ..., -0.0356071 ,
         -0.04697017, -0.04462998],
        [-0.01015817, -0.00232852,  0.02534184, ..., -0.01022941,
         -0.03510907,  0.00013487],
        ...,
        [-0.00586237,  0.01954675,  0.03310635, ...,  0.04467427,
         -0.01850789, -0.00245897],
        [-0.00586237,  0.01954675,  0.03310635, ...,  0.04467427,
         -0.01850789, -0.00245897],
        [-0.00586237,  0.01954675,  0.03310635, ...,  0.04467427,
         -0.01850789, -0.00245897]]], dtype=float32)>

In [29]:
train_sentences.shape

(6851,)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
model_0 = Pipeline([("tfidf",TfidfVectorizer()),
                    ("clf",MultinomialNB())])
model_0.fit(train_sentences,train_labels)
baseline_score= model_0.score(val_sentences,val_labels)
print(f" accuracy achieved by baseline model is {baseline_score*100:.2f}")

 accuracy achieved by baseline model is 79.27


In [31]:
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [32]:
# function to evaluate : accuracy,precisiion,recall,f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision, model_recall,model_f1, _=precision_recall_fscore_support(y_true,y_pred,average="weighted")
  model_results={"accuracy":model_accuracy,
                 "precision":model_precision,
                 "recall":model_recall,
                 "f1":model_f1}
  return model_results

In [33]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [34]:
 #model 1 : A simple dense model

In [35]:
from helper_functions import create_tensorboard_callback
SAVE_DIR="model_logs"


In [36]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
# compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [39]:
#Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name = "model_1_dense")])


Saving TensorBoard log files to: model_logs/model_1_dense/20230426-185737
Epoch 1/5
215/215 [==============================] - 28s 94ms/step - loss: 0.6126 - accuracy: 0.6903 - val_loss: 0.5375 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4428 - accuracy: 0.8184 - val_loss: 0.4700 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3472 - accuracy: 0.8578 - val_loss: 0.4590 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2852 - accuracy: 0.8902 - val_loss: 0.4644 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2381 - accuracy: 0.9107 - val_loss: 0.4812 - val_accuracy: 0.7782


In [40]:
# model_1_results = calculate_results(y_true=val_labels,
                                    #  y_pred=)
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4812 - accuracy: 0.7782


[0.48120468854904175, 0.778215229511261]

In [41]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

24/24 [==============================] - 0s 3ms/step


array([[4.43221003e-01],
       [7.56815076e-01],
       [9.97181058e-01],
       [1.58884615e-01],
       [1.04428500e-01],
       [9.42636907e-01],
       [9.08736765e-01],
       [9.91326034e-01],
       [9.70503271e-01],
       [3.63807201e-01],
       [1.23748124e-01],
       [6.98069096e-01],
       [6.24726601e-02],
       [1.76817372e-01],
       [4.70996602e-03],
       [1.56339705e-01],
       [2.30385028e-02],
       [7.65611008e-02],
       [2.97906160e-01],
       [5.86356580e-01],
       [9.15738821e-01],
       [4.47024591e-02],
       [4.30117279e-01],
       [1.02065653e-01],
       [9.58517313e-01],
       [9.98907208e-01],
       [4.50663678e-02],
       [8.73314366e-02],
       [2.72321105e-02],
       [1.94298014e-01],
       [6.18097246e-01],
       [3.61331344e-01],
       [5.03706992e-01],
       [1.76177919e-01],
       [5.51605225e-01],
       [6.95772171e-02],
       [9.95237231e-01],
       [2.04102740e-01],
       [3.35134342e-02],
       [9.98690903e-01],


In [42]:
# model_1_results = calculate_results(y_true=val_labels,
#                                      y_pred=model_1_pred_probs)
# model_1_results

In [43]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [44]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [45]:
model_1_results = calculate_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 77.82152230971128,
 'precision': 0.7810722984564322,
 'recall': 0.7782152230971129,
 'f1': 0.7757459581978545}

In [46]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [47]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [48]:
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'as',
 'up',
 'so',
 'just',
 'but',
 'me',
 'im',
 'your',
 'not',
 'amp',
 'out',
 'its',
 'will',
 'an',
 'no',
 'has',
 'fire',
 'after',
 'all',
 'when',
 'we',
 'if',
 'now',
 'via',
 'new',
 'more',
 'get',
 'or',
 'about',
 'what',
 'he',
 'people',
 'news',
 'been',
 'over',
 'one',
 'how',
 'dont',
 'they',
 'who',
 'into',
 'were',
 'do',
 'us',
 '2',
 'can',
 'video',
 'emergency',
 'there',
 'disaster',
 'than',
 'police',
 'would',
 'his',
 'still',
 'her',
 'some',
 'body',
 'storm',
 'crash',
 'burning',
 'suicide',
 'back',
 'man',
 'california',
 'why',
 'time',
 'them',
 'had',
 'buildings',
 'rt',
 'first',
 'cant',
 'see',
 'got',
 'day',
 'off',
 'our',
 'going',
 'nuclear',
 'know',
 'world',
 'bomb',
 'fires',
 'love',
 'killed',
 'go',
 'attack',
 'youtube',
 'dead

In [49]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [50]:
#can visualize the results in https://projector.tensorflow.org/

In [51]:
# RNN

In [52]:
# Model 2 : LSTM (long short tern momory)

In [53]:
# creating LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x =  text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(64 , return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
print(outputs.shape)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")

(None, 1)


In [54]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [55]:
# compiling the model
model_2.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [56]:
# fit the model
model_2_history = model_2.fit(train_sentences,train_labels,epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230426-185815
Epoch 1/5
215/215 [==============================] - 16s 57ms/step - loss: 0.2189 - accuracy: 0.9206 - val_loss: 0.5882 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1564 - accuracy: 0.9425 - val_loss: 0.5879 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1283 - accuracy: 0.9505 - val_loss: 0.6670 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1066 - accuracy: 0.9593 - val_loss: 0.6950 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0881 - accuracy: 0.9661 - val_loss: 0.9523 - val_accuracy: 0.7808


In [57]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[3.9657303e-03],
       [8.1837243e-01],
       [9.9972063e-01],
       [2.5050489e-02],
       [3.7788245e-04],
       [9.9686521e-01],
       [8.7215835e-01],
       [9.9977285e-01],
       [9.9968350e-01],
       [4.5943284e-01]], dtype=float32)

In [58]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [59]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred = model_2_preds)
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7834583238190046,
 'recall': 0.7808398950131233,
 'f1': 0.778533312750939}

In [60]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [61]:
# Model 3: GRU(gated recurrent unit)
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x =  text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64 , return_sequences=True)(x)
# x = layers.LSTM(64 , return_sequences=True)(x)
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
# print(outputs.shape)
model_3 = tf.keras.Model(inputs,outputs,name="model_3_GRU")

In [62]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [63]:
# copile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [64]:
# fit the model
model_3_history = model_3.fit(train_sentences,train_labels,epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230426-185900
Epoch 1/5
215/215 [==============================] - 16s 59ms/step - loss: 0.1500 - accuracy: 0.9400 - val_loss: 0.7224 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0807 - accuracy: 0.9692 - val_loss: 0.7881 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0704 - accuracy: 0.9707 - val_loss: 1.0553 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0606 - accuracy: 0.9761 - val_loss: 1.0254 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0536 - accuracy: 0.9771 - val_loss: 1.1342 - val_accuracy: 0.7756


In [65]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[4.2776903e-04],
       [7.8535157e-01],
       [9.9984300e-01],
       [1.2199429e-01],
       [8.8971748e-05],
       [9.9958819e-01],
       [7.2911733e-01],
       [9.9995971e-01],
       [9.9987209e-01],
       [7.1554065e-01]], dtype=float32)

In [66]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [67]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results
# baseline_results

{'accuracy': 77.55905511811024,
 'precision': 0.7777490986405654,
 'recall': 0.7755905511811023,
 'f1': 0.7733619560087615}

In [68]:
# model 4: bidirectional RNN 

In [69]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [70]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [71]:
#COMPILE
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [72]:
#fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectioanl")])

Saving TensorBoard log files to: model_logs/model_4_bidirectioanl/20230426-185926
Epoch 1/5
215/215 [==============================] - 18s 65ms/step - loss: 0.1018 - accuracy: 0.9717 - val_loss: 0.8826 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0520 - accuracy: 0.9775 - val_loss: 1.0689 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0481 - accuracy: 0.9788 - val_loss: 1.2645 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0438 - accuracy: 0.9796 - val_loss: 1.2905 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0426 - accuracy: 0.9793 - val_loss: 1.4308 - val_accuracy: 0.7769


In [73]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[2.6319540e-04],
       [7.6054776e-01],
       [9.9997318e-01],
       [3.0717519e-01],
       [9.1565125e-06],
       [9.9806195e-01],
       [8.2322127e-01],
       [9.9998760e-01],
       [9.9997091e-01],
       [9.0735829e-01]], dtype=float32)

In [74]:
model_4_preds = tf.squeeze(tf.squeeze(tf.round(model_4_pred_probs)))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [75]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.69028871391076,
 'precision': 0.7805885493126917,
 'recall': 0.7769028871391076,
 'f1': 0.774064384984916}

In [76]:
model_3_results

{'accuracy': 77.55905511811024,
 'precision': 0.7777490986405654,
 'recall': 0.7755905511811023,
 'f1': 0.7733619560087615}

In [77]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7834583238190046,
 'recall': 0.7808398950131233,
 'f1': 0.778533312750939}

In [78]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [79]:
#model 5 : Conv1D
# Create 1-D convolutional layer to modelsequences

In [80]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

#compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

#Get a summary of Conv1D model
model_5.summary()


Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_4 (Dense)             (None, 1)              

In [81]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230426-190012
Epoch 1/5
215/215 [==============================] - 17s 53ms/step - loss: 0.1186 - accuracy: 0.9594 - val_loss: 0.9485 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0742 - accuracy: 0.9715 - val_loss: 1.0369 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0628 - accuracy: 0.9752 - val_loss: 1.1440 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0550 - accuracy: 0.9772 - val_loss: 1.2210 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0521 - accuracy: 0.9784 - val_loss: 1.2339 - val_accuracy: 0.7520


In [82]:
model_5_pred_probs = model_5.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [83]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [84]:
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred = model_5_preds)
model_5_results

{'accuracy': 75.19685039370079,
 'precision': 0.7515814976569224,
 'recall': 0.7519685039370079,
 'f1': 0.7510820012604243}

In [85]:
#model 6: Tensorflow Hub Pretrained Sentence Encoder

In [88]:
sample_sentence = "There's a flood in my street"

In [89]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                       "universal sentence encoder turn sentence into numbers"])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [93]:
embed_samples[1].shape

TensorShape([512])

fatal: not a git repository (or any of the parent directories): .git
